## Exercise: Using LSTMs to Classify the 20 Newsgroups Data Set
The 20 Newsgroups data set is a well known classification problem. The goal is to classify which newsgroup a particular post came from.  The 20 possible groups are:

`comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x	rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey	
sci.crypt
sci.electronics
sci.med
sci.space
misc.forsale	
talk.politics.misc
talk.politics.guns
talk.politics.mideast	
talk.religion.misc
alt.atheism
soc.religion.christian`

As you can see, some pairs of groups may be quite similar while others are very different.

The data is given as a designated training set of size 11314 and test set of size 7532.  The 20 categories are represented in roughly equal proportions, so the baseline accuracy is around 5%.


To begin, review the code below.  This will walk you through the basics of loading in the 20 newsgroups data, loading in the GloVe data, building the word embedding matrix, and building the LSTM model.

After we build the first LSTM model, it will be your turn to build one and play with the parameters.

In [1]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

import keras

from sklearn.datasets import fetch_20newsgroups

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
max_features = 20000
seq_length = 30  # How long to make our word sequences
batch_size = 32



In [3]:
# Download the 20 newsgroups data - there is already a designated "train" and "test" set

newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')



In [4]:
len(newsgroups_train.data), len(newsgroups_test.data)

(11314, 7532)

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(newsgroups_train.data)

In [6]:
sequences_train = tokenizer.texts_to_sequences(newsgroups_train.data)
sequences_test = tokenizer.texts_to_sequences(newsgroups_test.data)

In [7]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 134142 unique tokens.


In [8]:
x_train = pad_sequences(sequences_train, maxlen=seq_length)
x_test = pad_sequences(sequences_test, maxlen=seq_length)



In [9]:
x_train

array([[ 2919,   198,     3, ...,    35,    58,  7872],
       [  351,   138,   534, ...,   118,   441,    15],
       [    9,    33,     4, ...,   187,    84, 17405],
       ..., 
       [    1,  1788,     3, ..., 19956,   383,    31],
       [  115,   362,    67, ...,  7773,   486,   493],
       [ 4476, 13858,  1031, ...,   200,    38,  3829]], dtype=int32)

In [10]:
y_train = keras.utils.to_categorical(np.asarray(newsgroups_train.target))
y_test = keras.utils.to_categorical(np.asarray(newsgroups_test.target))

We will be using the Glove pre-trained word vectors.  If you haven't already, please download them using this link:
(NOTE: this will start downloading an 822MB file)

http://nlp.stanford.edu/data/glove.6B.zip

Then unzip the file and fill your local path to the file in the code cell below.

We will use the file `glove.6B.100d.txt`

In [11]:
embeddings_index = {}
f = open('/Users/lucenator/Work/Data/glove/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Let's just look at a word embedding

In [12]:
dog_vec = embeddings_index['dog']
dog_vec

array([ 0.30816999,  0.30937999,  0.52802998, -0.92543   , -0.73671001,
        0.63475001,  0.44196999,  0.10262   , -0.09142   , -0.56607002,
       -0.5327    ,  0.2013    ,  0.77039999, -0.13982999,  0.13727   ,
        1.1128    ,  0.89301002, -0.17869   , -0.0019722 ,  0.57288998,
        0.59478998,  0.50427997, -0.28990999, -1.34909999,  0.42756   ,
        1.27479994, -1.16129994, -0.41084   ,  0.042804  ,  0.54865998,
        0.18897   ,  0.3759    ,  0.58034998,  0.66974998,  0.81155998,
        0.93864   , -0.51005   , -0.070079  ,  0.82819003, -0.35346001,
        0.21086   , -0.24412   , -0.16553999, -0.78358001, -0.48482001,
        0.38968   , -0.86356002, -0.016391  ,  0.31984001, -0.49246001,
       -0.069363  ,  0.018869  , -0.098286  ,  1.31260002, -0.12116   ,
       -1.23989999, -0.091429  ,  0.35293999,  0.64644998,  0.089642  ,
        0.70293999,  1.12440002,  0.38639   ,  0.52083999,  0.98786998,
        0.79952002, -0.34625   ,  0.14094999,  0.80167001,  0.20

In [13]:
## This creates a matrix where the $i$th row gives the word embedding for the word represented by integer $i$.
## Essentially, these will be the "weights" for the Embedding Layer
## Rather than learning the weights, we will use these ones and "freeze" the layer

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [14]:
embedding_matrix.shape

(134143, 100)

## LSTM Layer
`keras.layers.recurrent.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- Similar in structure to the `SimpleRNN` layer
- `units` defines the dimension of the recurrent state
- `recurrent_...` refers the recurrent state aspects of the LSTM
- `kernel_...` refers to the transformations done on the input



In [15]:
word_dimension = 100  # This is the dimension of the words we are using from GloVe
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                            word_dimension,  
                            weights=[embedding_matrix],  # We set the weights to be the word vectors from GloVe
                            input_length=seq_length,
                            trainable=False))  # By setting trainable to False, we "freeze" the word embeddings.
model.add(LSTM(30, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           13414300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                15720     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
Total params: 13,430,640.0
Trainable params: 16,340.0
Non-trainable params: 13,414,300.0
_________________________________________________________________


In [16]:
rmsprop = keras.optimizers.RMSprop(lr = .002)

model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [17]:

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))

Train on 11314 samples, validate on 7532 samples
Epoch 1/20
11314/11314 [==============================] - 28s - loss: 2.7819 - acc: 0.1419 - val_loss: 2.4770 - val_acc: 0.2144
Epoch 2/20
11314/11314 [==============================] - 26s - loss: 2.3775 - acc: 0.2546 - val_loss: 2.2147 - val_acc: 0.3002
Epoch 3/20
11314/11314 [==============================] - 27s - loss: 2.1631 - acc: 0.3333 - val_loss: 2.0576 - val_acc: 0.3595
Epoch 4/20
11314/11314 [==============================] - 26s - loss: 2.0165 - acc: 0.3706 - val_loss: 1.9689 - val_acc: 0.3802
Epoch 5/20
11314/11314 [==============================] - 26s - loss: 1.9147 - acc: 0.4103 - val_loss: 1.8981 - val_acc: 0.4110
Epoch 6/20
11314/11314 [==============================] - 27s - loss: 1.8380 - acc: 0.4344 - val_loss: 1.8604 - val_acc: 0.4294
Epoch 7/20
11314/11314 [==============================] - 27s - loss: 1.7718 - acc: 0.4577 - val_loss: 1.8264 - val_acc: 0.4331
Epoch 8/20
11314/11314 [==============================]

In [26]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))

Train on 11314 samples, validate on 7532 samples
Epoch 1/20
11314/11314 [==============================] - 27s - loss: 1.4687 - acc: 0.5475 - val_loss: 1.7580 - val_acc: 0.4705
Epoch 2/20
11314/11314 [==============================] - 29s - loss: 1.4530 - acc: 0.5498 - val_loss: 1.7694 - val_acc: 0.4724
Epoch 3/20
11314/11314 [==============================] - 27s - loss: 1.4414 - acc: 0.5527 - val_loss: 1.7539 - val_acc: 0.4756
Epoch 4/20
11314/11314 [==============================] - 27s - loss: 1.4429 - acc: 0.5552 - val_loss: 1.7551 - val_acc: 0.4758
Epoch 5/20
11314/11314 [==============================] - 27s - loss: 1.4363 - acc: 0.5542 - val_loss: 1.7455 - val_acc: 0.4801
Epoch 6/20
11314/11314 [==============================] - 27s - loss: 1.4051 - acc: 0.5628 - val_loss: 1.7586 - val_acc: 0.4858
Epoch 7/20
11314/11314 [==============================] - 26s - loss: 1.4047 - acc: 0.5628 - val_loss: 1.7568 - val_acc: 0.4866
Epoch 8/20
11314/11314 [==============================]

## Exercise
### Your Turn
- Build a neural network with a SimpleRNN instead of an LSTM (with other dimensions and parameters the same). How does the performance compare?
- Use the LSTM above without the pretrained word vectors (randomly initialize the weights and have them be learned during the training process).  How does the performance compare?
- Try different sequence lengths, and dimensions for the hidden state of the LSTM.  Can you improve the model?


In [18]:
from keras.layers import SimpleRNN
from keras import initializers


In [19]:
rnn_hidden_dim=50
model_2 = Sequential()
model_2.add(Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=seq_length,
                            trainable=False))
model_2.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu'))

model_2.add(Dense(20, activation='softmax'))

model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 100)           13414300  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 50)                7550      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
Total params: 13,422,870.0
Trainable params: 8,570.0
Non-trainable params: 13,414,300.0
_________________________________________________________________


In [20]:
rmsprop = keras.optimizers.RMSprop(lr = .0002)

model_2.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
          validation_data=(x_test, y_test))

Train on 11314 samples, validate on 7532 samples
Epoch 1/20
11314/11314 [==============================] - 6s - loss: 2.5975 - acc: 0.1904 - val_loss: 2.3919 - val_acc: 0.2270
Epoch 2/20
11314/11314 [==============================] - 5s - loss: 2.2637 - acc: 0.2911 - val_loss: 2.2491 - val_acc: 0.3071
Epoch 3/20
11314/11314 [==============================] - 5s - loss: 2.1317 - acc: 0.3366 - val_loss: 2.3488 - val_acc: 0.2608
Epoch 4/20
11314/11314 [==============================] - 5s - loss: 2.0498 - acc: 0.3645 - val_loss: 2.1709 - val_acc: 0.3331
Epoch 5/20
11314/11314 [==============================] - 5s - loss: 1.9899 - acc: 0.3850 - val_loss: 2.0976 - val_acc: 0.3478
Epoch 6/20
11314/11314 [==============================] - 5s - loss: 1.9385 - acc: 0.4003 - val_loss: 2.0430 - val_acc: 0.3789
Epoch 7/20
11314/11314 [==============================] - 5s - loss: 1.8990 - acc: 0.4151 - val_loss: 2.0833 - val_acc: 0.3666
Epoch 8/20
11314/11314 [==============================] - 5s -

In [23]:
model_3 = Sequential()
model_3.add(Embedding(len(word_index) + 1,
                            word_dimension))  # By setting trainable to False, we "freeze" the word embeddings.
model_3.add(LSTM(30, dropout=0.2, recurrent_dropout=0.2))
model_3.add(Dense(20, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           13414300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                15720     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
Total params: 13,430,640.0
Trainable params: 16,340.0
Non-trainable params: 13,414,300.0
_________________________________________________________________


In [24]:
rmsprop = keras.optimizers.RMSprop(lr = .002)

model_3.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_3.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))

Train on 11314 samples, validate on 7532 samples
Epoch 1/20
11314/11314 [==============================] - 95s - loss: 2.9882 - acc: 0.1176 - val_loss: 2.9809 - val_acc: 0.1581
Epoch 2/20
11314/11314 [==============================] - 94s - loss: 2.9401 - acc: 0.1635 - val_loss: 2.9067 - val_acc: 0.1240
Epoch 3/20
11314/11314 [==============================] - 93s - loss: 2.8293 - acc: 0.1461 - val_loss: 2.8121 - val_acc: 0.1459
Epoch 4/20
11314/11314 [==============================] - 93s - loss: 2.7022 - acc: 0.1818 - val_loss: 2.7187 - val_acc: 0.1928
Epoch 5/20
11314/11314 [==============================] - 94s - loss: 2.5835 - acc: 0.2302 - val_loss: 2.6297 - val_acc: 0.2375
Epoch 6/20
11314/11314 [==============================] - 93s - loss: 2.4713 - acc: 0.2636 - val_loss: 2.5360 - val_acc: 0.2589
Epoch 7/20
11314/11314 [==============================] - 94s - loss: 2.3612 - acc: 0.2984 - val_loss: 2.4737 - val_acc: 0.2704
Epoch 8/20
11314/11314 [==============================]